In [1]:
import pandas as pd
import numpy as np
import os
import csv
import datetime as d
from datetime import datetime as D
import matplotlib.ticker as ticker
import random 
import time
import math
import scipy
from scipy import stats
import gzip
import sys
import datetime
import ast

In [2]:
data_path = '../../../../../../../Volumes/Seagate_Qing/Safegraph/'
save_dir = '../../../../../../../Volumes/Seagate_Qing/Safegraph_clean/'
years = ['2020','2021','2022']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
#np.save('ym_datesofweek',ym_datesofweek)
ym_datesofweek = np.load('ym_datesofweek.npy',allow_pickle=True).item()

In [3]:
US_states = pd.read_csv('us-state-ansi-fips.csv')
state_lists_0 = US_states.stusps.values
state_lists = [s[1:] for s in state_lists_0]
state_lists.remove('HI')
state_lists.remove('AK') ### remove the non mainland
real_data_path = '/Users/qingyao/OneDrive - cumc.columbia.edu/machine_learning_data/'
fips_main = pd.read_csv(real_data_path+'county_data/fips_mainland.csv',names=['FIPS'])
fips_main_sorted = fips_main.sort_values(by='FIPS')
fips_main_sorted['poi_idx'] = fips_main_sorted.index

In [4]:
csv.field_size_limit(500 * 1024 * 1024)

131072

In [5]:
df_header = pd.read_csv('header.csv')
df_header.iloc[[32,34,36,38],0].values

array(['raw_visit_counts', 'visits_by_day', 'poi_cbg',
       'visitor_home_aggregation'], dtype=object)

In [6]:
selected_header = ['raw_visit_counts', 'visits_by_day', 'poi_cbg','visitor_home_aggregation']

In [ ]:
#### preprocess extract all the nan data
y = '2020'
for m in ['03']:
    print('month:', m)
    subdir = data_path + y + '/' + m + '/'
    week_start_dates = ym_datesofweek[y + m]
    for start_day in week_start_dates[:1]:
        print(start_day)
        weekly_folder = data_path + y + '/' + m + '/' + start_day + '/SAFEGRAPH/WP/'
        for i in range(1,30):  ### there are 40 files !!!
            file_name = weekly_folder + 'core_poi-geometry-patterns-part{}.csv.gz'.format(i)
            file_stat = os.stat(file_name)
            print(i)
            count = 0
            actual = 0
            new_name = save_dir + y + '_' + m + '_' + start_day + '_{}.csv'.format(i)
            with open(new_name, 'w', encoding='utf-8', newline='') as csv_out_file:  # Open the file for writing (not appending)
                filewriter = csv.writer(csv_out_file)
                filewriter.writerow(selected_header)  # Write the header once for each new file
                with gzip.open(file_name, 'rt') as csv_in_file:
                    filereader = csv.reader(csv_in_file)
                    for row_list in filereader:
                        count = count + 1
                        if (row_list[13] in state_lists) and (row_list[28] == 'US') and (row_list[34] != '') and (row_list[36] != '') and (row_list[38] != '{}'):  ### select us mainland first
                            actual = actual+1
                            filewriter.writerow([row_list[32], row_list[34], row_list[36], row_list[38]])
            print(actual/count)

In [7]:
%run ../codes/mobility_matrix_extraction.py

In [27]:
%timeit
#### extract the M
y = '2020'
for m in ['03']:
    print('month:', m)
    subdir = data_path + y + '/' + m + '/'
    week_start_dates = ym_datesofweek[y + m]
    for start_day in week_start_dates[:1]:
        print(start_day)
        weekly_folder = subdir + start_day + '/SAFEGRAPH/WP/'
        M_w = np.zeros((7,3108,3108)) #M_weekly = np.zeros((7,3108,3108)) ## 
        for i in range(1,2):  ### there are 40 files !!!
            print(i)
            new_name = save_dir + y + '_' + m + '_' + start_day + '_{}.csv'.format(i)
            df_file = pd.read_csv(new_name)
            df_file['poi_fips'] = df_file['poi_cbg'].astype(str).str[:-7].astype(int)
            df_i = pd.merge(df_file,fips_main_sorted,left_on='poi_fips',right_on='FIPS',how='left') 
            df_i_main = df_i.dropna()
            df_i_main['visits_by_day_list'] = df_i_main['visits_by_day'].apply(lambda x: np.array(ast.literal_eval(x)))
            df_i_main['multiplier'] = df_i_main.apply(lambda x: x['visits_by_day_list']/x['raw_visit_counts'], axis=1) ### multiplier
            df_i_main['visitor_home_list'] = df_i_main['visitor_home_aggregation'].apply(lambda x: x[1:-1].replace('"','').split(','))
            df_i_main[['home_idx_list', 'modified_visit_list']] = df_i_main.apply(lambda x: process_visitor_data(x['visitor_home_list'],x['raw_visit_counts'],fips_main_sorted),axis=1)
            df_i_main['vis_daily_matrix'] = df_i_main.apply(lambda x: dayvisits_by_visitor(x['modified_visit_list'], x['multiplier']),axis=1)
            for j_data_idx in range(len(df_i_main)):
                df_j = df_i_main.iloc[j_data_idx]
                M_w = mobility_extract_per_poi(df_j, M_w)

month: 03
02
1


In [17]:
def Getmodify(visitorHome,visitorNum):# modify bias between effective cencus tract data  and visitor num
    new_visitor_aray = np.zeros((visitorHome.shape[0]))
    visitor_aray = visitorHome.values
    homeSum = visitor_aray.sum()
    weight = visitor_aray/homeSum
    loss = visitorNum - homeSum
    new_visitor_aray = visitor_aray + loss*weight
    return new_visitor_aray

In [28]:
%timeit
#### extract the M
y = '2020'
for m in ['03']:
    print('month:', m)
    subdir = data_path + y + '/' + m + '/'
    week_start_dates = ym_datesofweek[y + m]
    for start_day in week_start_dates[:1]:
        print(start_day)
        weekly_folder = subdir + start_day + '/SAFEGRAPH/WP/'
        M_weekly = np.zeros((7,3108,3108)) ## 
        for i in range(1,2):  ### there are 40 files !!!
            print(i)
            new_name = save_dir + y + '_' + m + '_' + start_day + '_{}.csv'.format(i)
            with open(new_name, 'r', encoding='utf-8', newline='') as csv_in_file:  # Open the file for writing (not appending)
                filereader = csv.reader(csv_in_file)
                next(filereader)  # Skip the header (first row)
                for row_list in filereader:
                    destination = np.int64(row_list[2][:-7]) 
                    destination_idx = fips_main_sorted[fips_main_sorted['FIPS'] == destination].index.values[0]
#                     print('poi_idx', destination_idx)
                    listCT=[]# save cencus tract name
                    listCTvisitor=[]# save visitor num
        
                    visitor = int(row_list[0]) #raw_visit_counts
                    vitiorRaw = row_list[1].replace('[','').replace(']','')
                    listOfDailyVisitor = [int(i) for i in vitiorRaw.split(",")]
                    
                    DailyVisitor = np.array(listOfDailyVisitor)
                    dailyVisitorweight = DailyVisitor/visitor
                    
                    cbgRaw=row_list[3].replace('{','').replace('}','')
                    
                    filtered_cbgRaw = [item for item in cbgRaw.replace('"','').split(',') if not item.startswith('CA:')]

                    listOfcbg=[str(i.split(':')[0]) for i in filtered_cbgRaw]
                    listOfcbgnum=[int(i.split(':')[1]) for i in filtered_cbgRaw]
                    for i in range(len(listOfcbg)): ### for every home
                        listCT.append(listOfcbg[i])
                        listCTvisitor.append(listOfcbgnum[i])

                    CT = pd.DataFrame(listCT,dtype='str') ## make them the dataframe cbgs county fips
                    visitorHome = pd.DataFrame(listCTvisitor,dtype='int') ### numbers
                    modifyHomevisitor = Getmodify(visitorHome, visitor)
                    
                    for i, modified_visitor_count in enumerate(modifyHomevisitor):
                        home_location = CT.iloc[i][0]
                        home_county = np.int64(home_location[:-6])
                        visitor_count_by_day = modified_visitor_count * dailyVisitorweight
#                         print (home_county, visitor_count_by_day)
                        if home_county in fips_main_sorted['FIPS'].values and destination in fips_main_sorted['FIPS'].values:
                            home_idx = fips_main_sorted[fips_main_sorted['FIPS'] == home_county].index[0]
                            destination_idx = fips_main_sorted[fips_main_sorted['FIPS'] == destination].index[0]
                            for d in range(7):
                                M_weekly[d, destination_idx, home_idx] += visitor_count_by_day[d]

month: 03
02
1


In [29]:
%timeit
for i in range(1000000):
    pass

In [ ]:
#normalize by  people num
pop = np.loadtxt('pop_sorted.csv')
M_norm = M_weekly / pop[np.newaxis, :, np.newaxis]
for d in range(7):
    np.savetxt(save_dir+'M_{}{}.csv'.format(y,m), M_weekly[0],delimiter=",")

In [ ]:
#Smooth M                                    
# M_smooth=np.zeros((M_t,42,42))
# for t in range(M_t):
#     M_set7=M[max(0,t-3):min(M_t,t+4)]
#     M_smooth[t]=np.mean(M_set7,axis=0) 
                                     
#normalize by  people num
N_raw=pd.read_excel(data_path+'NYCpop.xlsx',usecols=['num'],dtype='int')
N_all=N_raw.values
M_norm=np.zeros((M_t,42,42))
for t in range(M_t):
    for i in range(42):
        M_norm[t][i]=M_smooth[t][i] / N_all[i]